# Letra e)

Vamos refazer a estimação do $\beta_{FE}$, $\beta_{GLS}$ e o Teste de Hausman controlando por novas variáveis.


In [33]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats

In [91]:
df = pd.read_stata("cornwell.dta")
df = df.set_index(["county", "year"], drop=True)
df

crmrte    prbarr   prbconv   prbpris  avgsen     polpc  \
county year                                                             
1      81    0.039885  0.289696  0.402062  0.472222    5.61  0.001787   
       82    0.038345  0.338111  0.433005  0.506993    5.59  0.001767   
       83    0.030305  0.330449  0.525703  0.479705    5.80  0.001836   
       84    0.034726  0.362525  0.604706  0.520104    6.89  0.001886   
       85    0.036573  0.325395  0.578723  0.497059    6.55  0.001924   
...               ...       ...       ...       ...     ...       ...   
197    83    0.015575  0.226667  0.480392  0.428571    7.77  0.001073   
       84    0.013662  0.204188  1.410260  0.372727   10.11  0.001109   
       85    0.013086  0.180556  0.830769  0.333333    5.96  0.001054   
       86    0.012874  0.112676  2.250000  0.244444    7.68  0.001088   
       87    0.014193  0.207595  1.182930  0.360825   12.23  0.001186   

              density      taxpc  west  central  ...  lpctymle   lpctmin  \
county year                                      ...                       
1      81    2.307159  25.697630     0        1  ... -2.433870  3.006608   
       82    2.330254  24.874252     0        1  ... -2.449038  3.006608   
       83    2.341801  26.451443     0        1  ... -2.464036  3.006608   
       84    2.346420  26.842348     0        1  ... -2.478925  3.006608   
       85    2.364896  28.140337     0        1  ... -2.497306  3.006608   
...               ...        ...   ...      ...  ...       ...       ...   
197    83    0.869048  18.905853     1        0  ... -2.538060  1.697597   
       84    0.872024  22.704754     1        0  ... -2.548068  1.697597   
       85    0.875000  24.123611     1        0  ... -2.561072  1.697597   
       86    0.880952  24.981979     1        0  ... -2.580968  1.697597   
       87    0.889881  25.952581     1        0  ... -2.601006  1.697597   

             clcrmrte  clprbarr  clprbcon  clprbpri  clavgsen   clpolpc  \
county year                                                               
1      81         NaN       NaN       NaN       NaN       NaN       NaN   
       82   -0.039376  0.154542  0.074143  0.071048 -0.003571 -0.011364   
       83   -0.235316 -0.022922  0.193987 -0.055326  0.036879  0.038413   
       84    0.136180  0.092641  0.140006  0.080857  0.172213  0.026930   
       85    0.051825 -0.108054 -0.043918 -0.045320 -0.050606  0.020199   
...               ...       ...       ...       ...       ...       ...   
197    83   -0.148666 -0.010969 -0.127018  0.164303  0.157158  0.149330   
       84   -0.131037 -0.104441  1.076927 -0.139610  0.263255  0.032795   
       85   -0.043091 -0.123000 -0.529178 -0.111704 -0.528454 -0.050473   
       86   -0.016311 -0.471524  0.996334 -0.310156  0.253549  0.031580   
       87    0.097525  0.611073 -0.642936  0.389407  0.465272  0.086072   

              cltaxpc     clmix  
county year                      
1      81         NaN       NaN  
       82   -0.032565  0.030857  
       83    0.061477 -0.244732  
       84    0.014670 -0.027331  
       85    0.047223  0.172125  
...               ...       ...  
197    83    0.070461  0.020250  
       84    0.183103  0.026842  
       85    0.060617 -0.366374  
       86    0.034964 -0.067911  
       87    0.038116 -0.114410  

[630 rows x 57 columns]

In [92]:
# Vamos nomear as variáveis
y_name = "lcrmrte"
x_names = ["lprbarr","lprbconv","lprbpris","lavgsen","lpolpc","lwcon","lwser"]
id_col = "county"
time_col = "year"


In [93]:
# Verificar se há MultiIndex => transformar
if isinstance(df.index, pd.MultiIndex):
    df = df.reset_index()

In [94]:
# manter só colunas necessárias e remover missings
required = [id_col, time_col, y_name] + x_names
required

['county',
 'year',
 'lcrmrte',
 'lprbarr',
 'lprbconv',
 'lprbpris',
 'lavgsen',
 'lpolpc',
 'lwcon',
 'lwser']

In [95]:
df[required]

,county,year,lcrmrte,lprbarr,lprbconv,lprbpris,lavgsen,lpolpc,lwcon,lwser
0,1,81,-3.221757,-1.238923,-0.911149,-0.750306,1.724551,-6.327340,5.330205,5.374044
1,1,82,-3.261134,-1.084381,-0.837006,-0.679258,1.720979,-6.338704,5.360137,5.444911
2,1,83,-3.496449,-1.107303,-0.643019,-0.734584,1.757858,-6.300291,5.392628,5.481292
3,1,84,-3.360270,-1.014662,-0.503013,-0.653727,1.930071,-6.273361,5.409070,5.531204
4,1,85,-3.308445,-1.122715,-0.546931,-0.699047,1.879465,-6.253162,5.496169,5.564850
...,...,...,...,...,...,...,...,...,...,...
625,197,83,-4.162107,-1.484273,-0.733153,-0.847299,2.050270,-6.837371,5.333205,5.280478
626,197,84,-4.293144,-1.588714,0.343774,-0.986909,2.313525,-6.804576,5.439068,5.261022
627,197,85,-4.336235,-1.711714,-0.185403,-1.098613,1.785071,-6.855049,5.533534,5.333961
628,197,86,-4.352546,-2.183239,0.810930,-1.408769,2.038620,-6.823469,5.636770,5.371892


In [96]:
df = df[required].dropna().copy()   # dropna para simplicidade

In [98]:
df = df.sort_values([id_col, time_col]).reset_index(drop=True)

In [99]:
df

,county,year,lcrmrte,lprbarr,lprbconv,lprbpris,lavgsen,lpolpc,lwcon,lwser
0,1,81,-3.221757,-1.238923,-0.911149,-0.750306,1.724551,-6.327340,5.330205,5.374044
1,1,82,-3.261134,-1.084381,-0.837006,-0.679258,1.720979,-6.338704,5.360137,5.444911
2,1,83,-3.496449,-1.107303,-0.643019,-0.734584,1.757858,-6.300291,5.392628,5.481292
3,1,84,-3.360270,-1.014662,-0.503013,-0.653727,1.930071,-6.273361,5.409070,5.531204
4,1,85,-3.308445,-1.122715,-0.546931,-0.699047,1.879465,-6.253162,5.496169,5.564850
...,...,...,...,...,...,...,...,...,...,...
625,197,83,-4.162107,-1.484273,-0.733153,-0.847299,2.050270,-6.837371,5.333205,5.280478
626,197,84,-4.293144,-1.588714,0.343774,-0.986909,2.313525,-6.804576,5.439068,5.261022
627,197,85,-4.336235,-1.711714,-0.185403,-1.098613,1.785071,-6.855049,5.533534,5.333961
628,197,86,-4.352546,-2.183239,0.810930,-1.408769,2.038620,-6.823469,5.636770,5.371892


In [100]:
# Número de unidades
N = df[id_col].nunique()
T_mode = int(df.groupby(id_col)[time_col].nunique().mode().iloc[0])
K = len(x_names)

print(f"N = {N}, T (mode) = {T_mode}, K = {K}")
print("Observações após dropna:", df.shape[0])

N = 90, T (mode) = 7, K = 7
Observações após dropna: 630


In [101]:
# calcular médias por county e construir demeaned
means = df.groupby(id_col)[[y_name] + x_names].transform("mean")
df_within = df[[y_name] + x_names] - means

In [103]:
df_within

,lcrmrte,lprbarr,lprbconv,lprbpris,lavgsen,lpolpc,lwcon,lwser
0,0.112938,-0.110579,-0.232672,-0.012029,-0.110884,-0.032213,-0.124182,-0.141251
1,0.073562,0.043963,-0.158529,0.059018,-0.114456,-0.043577,-0.094249,-0.070384
2,-0.161753,0.021041,0.035458,0.003693,-0.077577,-0.005164,-0.061759,-0.034002
3,-0.025574,0.113682,0.175464,0.084550,0.094636,0.021766,-0.045316,0.015910
4,0.026251,0.005629,0.131546,0.039230,0.044030,0.041965,0.041782,0.049556
...,...,...,...,...,...,...,...,...
625,0.043260,0.208983,-0.712753,0.210483,-0.094545,0.039442,-0.119931,0.022176
626,-0.087777,0.104542,0.364174,0.070873,0.168710,0.072237,-0.014068,0.002720
627,-0.130868,-0.018458,-0.165004,-0.040831,-0.359745,0.021764,0.080398,0.075660
628,-0.147179,-0.489982,0.831330,-0.350987,-0.106196,0.053344,0.183634,0.113590


In [60]:
# montar e rodar OLS sem constante
y_within = df_within[y_name].values
X_within = df_within[x_names].values  # ordem = x_names
fe_ols = sm.OLS(y_within, X_within).fit()

print(fe_ols.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.372
Model:                            OLS   Adj. R-squared (uncentered):              0.365
Method:                 Least Squares   F-statistic:                              52.78
Date:                Wed, 03 Dec 2025   Prob (F-statistic):                    4.51e-59
Time:                        14:12:14   Log-Likelihood:                          372.85
No. Observations:                 630   AIC:                                     -731.7
Df Residuals:                     623   BIC:                                     -700.6
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [68]:
beta_fe = fe_ols.params # Parâmetros(betas estimados)
V_fe = fe_ols.cov_params() #Matriz de covariância dos betas estimados
se_fe = np.sqrt(np.diag(V_fe))

print("\n=== FE (within) ===")
for nm, b, s in zip(x_names, beta_fe, se_fe):
    print(f"{nm:6s}  coef = {b:10.4f}   se = {s:8.4f}")


=== FE (within) ===
lprbarr  coef =    -0.3888   se =   0.0308
lprbconv  coef =    -0.3095   se =   0.0201
lprbpris  coef =    -0.2093   se =   0.0308
lavgsen  coef =     0.0332   se =   0.0240
lpolpc  coef =     0.4197   se =   0.0253
lwcon   coef =    -0.0990   se =   0.0305
lwser   coef =    -0.0146   se =   0.0180


In [76]:
# Estimar sigma_e^2 via residuos within
resid_within = fe_ols.resid # resíduos da regressão
df_dot_e = N*(T_mode - 1) - K
sigma_e2 = (resid_within**2).sum() / df_dof_e
sigma_e2 = float(sigma_e2)
print(f"\nsigma_e^2 (from FE residuals) = {sigma_e2:.6g}")


sigma_e^2 (from FE residuals) = 0.0211878


In [78]:
# BETWEEN estimator (médias por county) para obter sigma_a^2

group_means = df.groupby(id_col)[[y_name] + x_names].mean()
Y_bar = group_means[y_name]
X_bar = group_means[x_names]
Xb = sm.add_constant(X_bar)
be = sm.OLS(Y_bar, Xb).fit()
resid_between = be.resid

df_dof_a = N - (K + 1)   # K slopes + intercept
SSR_between = (resid_between**2).sum()
sigma_a2 = SSR_between / df_dof_a - sigma_e2 / T_mode
sigma_a2 = max(0.0, float(sigma_a2))
print(f"sigma_a^2 (estimated from BETWEEN) = {sigma_a2:.6g}")

sigma_a^2 (estimated from BETWEEN) = 0.0903135


In [79]:
# Cálculo de lambda (λ̂) e transformar (GLS quasi)
den = sigma_e2 + T_mode * sigma_a2
lam = 0.0 if den <= 0 else 1.0 - np.sqrt(sigma_e2 / den)
print(f"lambda_hat = {lam:.6g}")

lambda_hat = 0.819923


In [82]:
means_expanded = df.groupby(id_col)[[y_name] + x_names].transform("mean")
y_re = df[y_name] - lam * means_expanded[y_name]
X_re = df[x_names] - lam * means_expanded[x_names]
X_re_const = sm.add_constant(X_re)

In [84]:
# RE (GLS manual via OLS nas variáveis transformadas)
re_ols = sm.OLS(y_re, X_re_const).fit()
# extrair apenas os coeficientes de slope (mesma ordem x_names)
beta_re = re_ols.params[x_names].values
V_re_full = re_ols.cov_params()
# obter cov matrix das slopes (exclui constante)
if hasattr(V_re_full, "loc"):
    V_re = V_re_full.loc[x_names, x_names].values
else:
    # fallback: assume const is primeira coluna
    V_re = V_re_full[1:,1:]

se_re = np.sqrt(np.diag(V_re))

print("\n=== RE (GLS manual) ===")
for nm, b, s in zip(x_names, beta_re, se_re):
    print(f"{nm:6s}  coef = {b:10.4f}   se = {s:8.4f}")


=== RE (GLS manual) ===
lprbarr  coef =    -0.4528   se =   0.0326
lprbconv  coef =    -0.3495   se =   0.0214
lprbpris  coef =    -0.1984   se =   0.0349
lavgsen  coef =     0.0256   se =   0.0274
lpolpc  coef =     0.4224   se =   0.0270
lwcon   coef =    -0.0726   se =   0.0346
lwser   coef =    -0.0144   se =   0.0205


In [86]:
# Hausman test (FE vs RE) - usando slopes comuns (x_names)
#    H = (b_FE - b_RE)' [Var(b_FE)-Var(b_RE)]^{-1} (b_FE - b_RE)
# ---------------------------
# garantir matrizes numpy
V_fe_arr = np.asarray(V_fe)
V_re_arr = np.asarray(V_re)
# alinhar dimensões (devem ser k x k)
if V_fe_arr.shape != V_re_arr.shape:
    raise ValueError("Mismatch in covariance matrix shapes between FE and RE")


d = beta_fe - beta_re
V_diff = V_fe_arr - V_re_arr

In [87]:
# inverter (usar pseudo-inverso se singular)
try:
    invV = np.linalg.inv(V_diff)
    used_pinv = False
except np.linalg.LinAlgError:
    invV = np.linalg.pinv(V_diff)
    used_pinv = True

H = float(d.T @ invV @ d)
df_chi = len(d)
p_value = stats.chi2.sf(H, df_chi)

print("\n=== Hausman test (FE vs RE) ===")
print(f"k = {df_chi}")
print(f"H = {H:.4f}")
print(f"p-value = {p_value:.6f}")
if used_pinv:
    print("Nota: matriz Var(FE)-Var(RE) sing. => usado pseudo-inverso.")

if p_value < 0.05:
    print("Conclusão: rejeitamos H0 ao nível 5% -> há evidência de correlação entre α_i e X_i (usar FE).")
else:
    print("Conclusão: não rejeitamos H0 ao nível 5% -> não há evidência forte de correlação (RE pode ser usado).")


=== Hausman test (FE vs RE) ===
k = 7
H = -66.0939
p-value = 1.000000
Conclusão: não rejeitamos H0 ao nível 5% -> não há evidência forte de correlação (RE pode ser usado).
